# # Analiza powodzi w Opolu (2024) z użyciem danych Sentinel-1 SAR

### Cel:
 - Załaduj dane Sentinel-1 VV sprzed i po powodzi
 - Przekształć intensywność radaru do jednostki dB
 - Oblicz różnicę między obrazami
 - Wydziel obszary z dużym spadkiem VV (potencjalne strefy zalania)

## Krok 1: Import Earth Engine i geemap

In [ ]:
import ee
import geemap

# ee.Authenticate()
ee.Initialize(project="ee-zuzannaslobodzian2")

In [5]:
region = ee.Geometry.Point([17.931, 50.675])

## Krok 2: Załaduj obrazy Sentinel-1 VV (przed i po powodzi)

In [11]:
image_before = (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(region)
            .filterDate('2024-08-01', '2024-08-15')
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .select('VV')
            .mean())

image_after = (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(region)
            .filterDate('2024-10-01', '2024-10-15')
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .select('VV')
            .mean())

## Krok 3: Przekształć dane do skali dB

In [12]:
dB_before = image_before.log10().multiply(10)
dB_after = image_after.log10().multiply(10)

## Krok 4: Oblicz różnicę i wydziel obszary potencjalnej powodzi

In [13]:
diff = dB_before.subtract(dB_after)
flood_mask = diff.gt(3)

## Krok 5: Wizualizacja wyników

In [15]:
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
vis_params = {'min': -25, 'max': 0}

Map.addLayer(dB_before, vis_params, 'before the flood')
Map.addLayer(dB_after, vis_params, 'after the flood')
Map.addLayer(diff, {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}, 'difference')
Map.addLayer(flood_mask.selfMask(), {'palette': ['cyan']}, 'potential flood')

Map

Map(center=[50.675, 17.931], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…